In [ ]:
import apache_beam as beam
from apache_beam.runners.interactive import interactive_runner
from apache_beam.runners.interactive.interactive_beam import *
from apache_beam.transforms import trigger
from apache_beam.options import pipeline_options
topic = "projects/google.com:clouddfe/topics/shakespeare-kinglear"

In [ ]:
options = pipeline_options.PipelineOptions()
options.view_as(pipeline_options.StandardOptions).streaming = True
p = beam.Pipeline(interactive_runner.InteractiveRunner(), options=options)
words = p | "read" >> beam.io.ReadFromPubSub(topic=topic)

In [ ]:
windowed_words = (words 
                  | "window" >> beam.WindowInto(
                      beam.window.FixedWindows(10),
                      trigger=trigger.AfterWatermark(),
                      accumulation_mode=trigger.AccumulationMode.DISCARDING))

In [ ]:
windowed_word_counts = (windowed_words
                        | "count" >> beam.combiners.Count.PerElement())
show(windowed_word_counts)

In [ ]:
show(windowed_word_counts)

In [ ]:
counts_lower = (windowed_word_counts 
                | beam.MapTuple(lambda w, c: (w.lower(), c))
                | beam.GroupByKey() 
                | beam.MapTuple(lambda w, c: (w, sum(c))))
show(counts_lower)